In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import itertools
import math

from functools import reduce

spm = pd.read_csv('SPM/SPM_results.csv', sep=',')
scc_col = pd.read_csv('SCC/SCC_COL_results.csv', sep=',')
scc_fb = pd.read_csv('SCC/SCC_FB_results.csv', sep=',')
sorting = pd.read_csv('sorting/sorting_results.csv', sep=',')
prefix_sum = pd.read_csv('prefix_sum/prefix_sum_results.csv', sep=',')
synthesis = pd.read_csv('synthesis/synthesis_results.csv', sep=',')

all_data = [spm, scc_col, scc_fb, sorting, prefix_sum, synthesis];
cols = list(map((lambda d : d.columns), all_data));
assert (all([all(cols[i-1] == cols[i]) for i in range(1, len(cols))]));

data = pd.concat(all_data, ignore_index=True);

# timeout_as_3_min = data;
# timeout_as_3_min["runtime"] = timeout_as_3_min["runtime"].replace("timeout", "18000"); # factor 10 less to compensate for only 1 run
# timeout_as_3_min["runtime"] = timeout_as_3_min["runtime"].astype(float);
# timeout_as_3_min[timeout_as_3_min["algorithm"] == "SCC_FB"]["runtime"].sum() / 360000 # here we multiply by 10 again
#data[data["algorithm"] == "SPM"]["runtime"].value_counts()



In [ ]:
data = data.drop(data[data["runtime"] == "timeout"].index);
data["runtime"] = data["runtime"].astype(float);
data["problem_size1"] = data["problem_size1"].astype(int);
data["problem_size2"] = data["problem_size2"].astype(int);
data["problem_size3"] = data["problem_size3"].astype(int);
data["weak_non_racing"] = data["weak_non_racing"].astype(bool);

data["problem_size"] = data[["problem_size1", "problem_size2", "problem_size3"]].max(axis=1);
data["name"] = data["algorithm"];
data["name"] = data["name"].replace("SPM", "Small Progress Measures")
data["name"] = data["name"].replace("SCC_FB", "Forward-Backward SCC")
data["name"] = data["name"].replace("SCC_COL", "Colouring/Heads-off")
data["name"] = data["name"].replace("synthesis", "Supervisory Controller Synthesis")
data["name"] = data["name"].replace("prefix_sum", "Prefix Sum")


In [ ]:
def bar_chart(all_rows, rows, title, variable_parameter, ordering=None, yscale="log", xlabel="Problem size", xscale="linear"):
    if ordering == None:
        ordering = all_rows[variable_parameter].unique();
    assert (len(ordering) == len(all_rows[variable_parameter].unique()))
    
    plt.figure();
    plt.title(title);
    rows = get_comparable_rows(all_rows, rows, variable_parameter);
    tot_width = 0.8;
    width = tot_width / len(ordering);
    offset = - 0.5 * tot_width + 0.5 * width;
    
    ys = [];
    p_sizes = list(rows["problem_size"].unique());
    p_sizes.sort();
    nrof_values = len(p_sizes);
    
    for o, rows in [(o, rows[rows[variable_parameter] == o]) for o in ordering]:
        Y = rows.groupby("problem_size")["runtime"].mean();
        assert(nrof_values == len(Y.values));
        X = [x + offset for x in range(len(Y.values))];
        ys.append(Y);
        plt.bar(X, Y.values, width=width, label=o);
        offset += width;
    plt.xlabel(xlabel);
    plt.xscale(xscale);
    plt.xticks(range(nrof_values), [str(round(p, -3)/1000)+"K" for p in p_sizes]);
    plt.yscale(yscale);
    plt.ylabel("runtime (ms)");
    plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left");
    plt.show();
    return ys;

def scatter_chart(all_rows, rows, title, variable_parameter, ordering=None, yscale="log", xlabel="Problem size", xscale="linear"):
    if ordering == None:
        ordering = all_rows[variable_parameter].unique();
    assert (len(ordering) == len(all_rows[variable_parameter].unique()))
    
    plt.figure();
    plt.title(title);
    rows = get_comparable_rows(all_rows, rows, variable_parameter);
    cols = ["C0", "C1", "C2", "C3", "C4", "C5"];
    col_idx = 0;
    for o, rows in [(o, rows[rows[variable_parameter] == o]) for o in ordering]:
        for c, rows in rows.groupby("config"):
            plt.plot(rows["problem_size"], rows["runtime"], color=cols[col_idx % len(cols)], label=o);
        col_idx += 1;
    
    plt.xlabel(xlabel);
    plt.xscale(xscale);
    plt.yscale(yscale);
    plt.ylabel("runtime (ms)");
    patches = [mpatches.Patch(color=cols[i % len(cols)], label=ordering[i]) for i in range(len(ordering))];
    plt.legend(handles=patches, bbox_to_anchor=(1.04, 1), loc="upper left");
    plt.show();

In [ ]:
def memorder_chart(algs, data):
    colors = {
        "relaxed": "C0",
        "acqrel": "C2",
        "seqcons": "C1"
    };
    linestyles = {
        True: "-",
        False: "--"
    };
    markers = {
        "graph": ("P", 7),
        "in-kernel": ("D", 5),
        "on-host": ("*", 8)
    };
    
    config_cols = [
        "schedule",
        "voting-strat"
    ];
    
    voting = [
        ("in-kernel", "in-kernel-simple"),
        ("on-host", "on-host-simple"),
        ("graph", "graph-simple")
    ];
    order = ["relaxed", "acqrel", "seqcons"];
    
    
    fig = plt.figure(constrained_layout=True, figsize=(10, 2.5 * len(algs)));
    subfigs = fig.subfigures(nrows=len(algs), ncols=1)
    
#    fig, axs = plt.subplots(len(algs), 3, figsize=(12, 3 * len(algs)), sharey=True);
#     plt.suptitle((
#         f"The relative runtime performance of each memory order per schedule strategy\n"
#         f"({alg}{f': {p_type}' if not p_type.startswith('Random') else ''})"
#     ), y=1.1);
    
    for a, subfig in enumerate(subfigs):
        (alg, p_type) = algs[a];
        rows = data[(data["algorithm"] == alg) & (data["problem_type"] == p_type)];
        name = rows["name"].unique()[0];
        sub_name = f" ({p_type})" if alg == "SPM" else "";
        subfig.suptitle(f'{name}{sub_name}:');
        axs = subfig.subplots(nrows=1, ncols=3, sharey=True);
        
        
        prepped = pd.concat([
            rows[
                (rows["schedule"] == s) &
                (rows["voting-strat"] == v) &
                (rows["memorder"] == o) &
                (rows["weak_non_racing"] == opt)
            ][["problem_size", "runtime"]]
            .set_index("problem_size")
            .rename(columns={"runtime": f"rt_{s}_{o}_{opt}"})
            for ((s, v), o, opt) in itertools.product(voting, order, [True, False])
        ], axis = 1);

        axs[0].set_ylabel("Relative runtime (%)");
        for ((sf, (s, v)), o, opt) in itertools.product(enumerate(voting), order, [True, False]):
            Y = (prepped[f"rt_{s}_{o}_{opt}"] / prepped[f"rt_{s}_relaxed_True"]) * 100;
            Y.sort_index(inplace=True);
            axs[sf].plot(
                Y.index,
                Y,
                label=f"{o}{' (optimised non-racing parameters)' if opt else ''}",
                color=colors[o],
                linestyle=linestyles[opt]
            );

            axs[sf].grid(True);
            axs[sf].set_xlim(10**3, 10**7);
            axs[sf].set_xscale("log");

            if a == 0:
                axs[sf].set_title(s);

            if a == len(algs)-1:
                axs[sf].set_xlabel("Problem size");
    
    patches = [
        Line2D(
            [0],
            [0],
            color=colors[o],
            linestyle=linestyles[opt],
            label=f"{o}{' (ONRP)' if opt else ''}",
            linewidth=1
        ) for (o, opt) in itertools.product(order, [False, True])
    ];
    
    fig.legend(handles=patches, bbox_to_anchor=(1.19, 0.97));

    def prune(name):
        return name.replace(" ", "-").replace("/", "-");
    
    plt.savefig(f"memorder-impact.svg", bbox_inches = 'tight');
    plt.show();


In [ ]:
def schedule_chart(alg, p_type, data, relative):
    rows = data[(data["algorithm"] == alg) & (data["problem_type"] == p_type)];
    voting = sorted(list(set(rows.groupby(["schedule", "voting-strat"]).groups.keys())));
    prepped = pd.concat([
        rows[
            (rows["schedule"] == s) &
            (rows["voting-strat"] == v) &
            (rows["memorder"] == "relaxed") &
            (rows["weak_non_racing"] == True)
        ][["problem_size", "runtime"]]
        .set_index("problem_size")
        .rename(columns={"runtime": f"rt_{s}_{v}"})
        for (s, v) in voting
    ], axis = 1);
    
    colors = {
        "graph": "C0",
        "on-host": "C1",
        "in-kernel": "C2",
    };
    
    marker = {
        "graph-simple": ("*", 8, "C3"),
        "in-kernel-simple": ("*", 8, "C4"),
        "on-host-simple": ("*", 8, "C5"),
        "on-host-alternating": ("D", 5, "C6"),
        "in-kernel-alternating": ("D", 5, "C7"),
        "graph-shared": ("+", 8, "C8"),
        "graph-shared-banks": ("x", 8, "C9"),
        
    };
    
    fig, ax = plt.subplots(figsize=(6, 4));
    if relative:
        plt.suptitle((
            f"The relative performance of schedule and voting strategies\n"
            f"({alg}{f': {p_type}' if not p_type.startswith('Random') else ''})"
        ), y=1.07);
        ax.set_ylabel("Relative runtime (%)");
    else:
        plt.suptitle((
            f"The performance of schedule and voting strategies compared\n"
            f"({alg}{f': {p_type}' if not p_type.startswith('Random') else ''})"
        ), y=1.07);
        ax.set_ylabel("Runtime (ms)");
        ax.set_yscale("log");
    
    ax.grid(True);
    ax.set_xlim(10**3, 10**7);
    ax.set_xscale("log");
    ax.set_xlabel("Problem size");
    
    for (s, v) in voting:
        if relative:
            Y = (prepped[f"rt_{s}_{v}"] / prepped[f"rt_graph_graph-simple"]) * 100;
        else:
            Y = prepped[f"rt_{s}_{v}"];
        (m, ms, mc) = marker[v];
        ax.plot(
            prepped.index,
            Y,
            label=v,
            color=colors[s],
            marker=m,
            markersize=ms,
            linestyle=':',
            markerfacecolor=mc,
            markeredgecolor=mc
        );
    
    plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left");
    plt.show();

In [ ]:
algs = [
    ("synthesis", "Random"),
    ("prefix_sum", "Random"),
    ("SPM", "Invariantly Inevitably Eat"),
    ("SPM", "Invariantly Plato Starves"),
    ("SCC_FB", "Random Graph (p=1.3/n)"),
    ("SCC_COL", "Random Graph (p=1.3/n)")
    #("sorting", "Random")
]
memorder_chart(algs, data);
# for (alg, p_type) in algs:
#     memorder_chart(alg, p_type, data);
#     schedule_chart(alg, p_type, data, True);

In [ ]:
config_parameters = [
    ("memorder", ["relaxed", "acqrel", "seqcons"]),
    ("voting-strat", ["naive-alternating", "naive"]),
    ("weak_read_only", [1, 0]),
    ("division_strat", ["gridsize", "blocksize"])
];
    
multi_indices = [
    pd.MultiIndex.from_tuples(
        [("", "algorithm"), ("", "problem_type"), ("", "problem_size")]
    )
];
for (c, opts) in config_parameters:
    multi_indices.append(
        pd.MultiIndex.from_product(
            [
                [c],
                opts + ["#configs"]
            ],
            names=['optimisation technique', 'variant']
        )
    )

table_columns = reduce(lambda x, y : x.append(y), multi_indices);

table_data = [];
runtime_cols = [];
config_cols = [];
cols_done = False;
for alg, alg_rows in data.groupby("algorithm"):
    for p_type, p_type_rows in alg_rows.groupby("problem_type"):
        for p_size, p_size_rows in p_type_rows.groupby("problem_size"):
            current_row = [alg, p_type, p_size];
            for (c, opts) in config_parameters:
                if not cols_done:
                    runtime_cols += list(range(len(current_row), len(current_row)+len(opts)));
                    config_cols += [len(current_row)+len(opts)];
                rows = get_comparable_rows(data, p_size_rows, c);
                nrof_comp_rows = len(rows.index);
                if nrof_comp_rows == 0:
                    current_row += ["-"] * len(opts);
                else:
                    mean_runtimes = [rows[rows[c] == o]["runtime"].mean() for o in opts];
                    relative_runtimes = [mean_runtimes[i]/mean_runtimes[0] for i in range(len(mean_runtimes))];
                    as_percentage = ["{0:.0%}".format((f-1)) for f in relative_runtimes];
                    current_row += as_percentage;
                current_row += [int(nrof_comp_rows/len(opts))];
            table_data.append(current_row);
            cols_done = True;
runtime_cols = [table_columns[i] for i in runtime_cols];
config_cols = [table_columns[i] for i in config_cols];

table = pd.DataFrame(data=table_data, columns = table_columns);


In [ ]:
def color_performance(val):
    if val == "0%":
        color = "black";
    elif val.startswith("-"):
        color = "green";
    else:
        color = "red";

    return 'color: {}'.format(color)

def border(val):
    return 'border-right: 1px solid black';

styled_table = table.style.applymap(color_performance, 
                  subset=runtime_cols).applymap(border, subset= [("", "problem_size")] + config_cols);

styled_table.to_excel(r'results_table.xlsx', index=True)
